# 探测器构建

G4 探测器构建顺序


几何体 -> 逻辑体 -> 物理体

  |
  
  形状




## 构造几何体





## 构造逻辑体



## 构造物理体




## 物理体之间的层级结构


**问题：如何将物理体A放入物理体B的内部？**

- 物理体A：Si
- 物理体B：C

**物理体A与物理体B的重叠部分：？**


Geant4:嵌套模型(盒子模型)

先定义母体 B，其材质为碳；再定义子体 A，其材质为硅，并声明其母体为 B。那么两者重叠区域材质最终由子体确定。


G4中，所有的物理体必须被指定母体。对于最外层的物理体，其母体为空，此物理体称为“World”。


